In [ ]:
### Train last FC layer of CNN (this is our feature extractor) - train using
### simple 9-class problem

In [5]:
class dataset_h5(torch.utils.data.Dataset):
  def __init__(self, data_type, hdf5_path):
      super(dataset_h5, self).__init__()

      self.hdf5_path = hdf5_path
      self.d_type = data_type
      with h5py.File(self.hdf5_path, 'r') as file:
          self.n_images = file[f'{self.d_type}_images'].shape[0]


  def __getitem__(self, index):
    with h5py.File(self.hdf5_path, 'r') as file:
        image = file[f'{self.d_type}_images'][index,:,:]
        label = file[f'{self.d_type}_labels'][index]
        # Convert to PyTorch tensor
        image = torch.tensor(image, dtype=torch.float32)
        return image, label

  def __len__(self):
      return self.n_images

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch.nn.functional as F
import torch
import pandas as pd
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchvision import models
import h5py
import numpy as np
from PIL import Image
import os
import math

In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [7]:
path_train = '/content/drive/MyDrive/data/train/'
path_val = '/content/drive/MyDrive/data/val/'
path_test = '/content/drive/MyDrive/data/test/'
BATCH_SIZE = 128
preprocess = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406],  std=[0.229, 0.224, 0.225])])
preprocess_tr = transforms.Compose([transforms.Resize(256), transforms.RandomRotation(degrees=20),transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406],  std=[0.229, 0.224, 0.225])])
train_dataset = ImageFolder(path_train, preprocess_tr)
val_dataset = ImageFolder(path_val, preprocess)

In [8]:
train_dataset[0][0].shape

torch.Size([3, 224, 224])

In [9]:
len(train_dataset), len(val_dataset)

(10504, 3502)

In [16]:
val_images = np.array([val_dataset[i][0] for i in range(len(val_dataset))])
val_labels = np.array([val_dataset[i][1] for i in range(len(val_dataset))])
print(val_images.shape)
print(val_labels.shape)

(3502, 3, 224, 224)
(3502,)


In [18]:
image_chunk_shape = (BATCH_SIZE,3,224,224)
label_chunk_shape = (BATCH_SIZE)
with h5py.File('full_images.hdf5', 'w') as hf:
    dset_img_val = hf.create_dataset('val_images', data=val_images, shape=(len(val_dataset), 3, 224, 224), chunks=image_chunk_shape)
    dset_label_val = hf.create_dataset('val_labels', data=val_labels, shape=(len(val_dataset)), chunks=label_chunk_shape)
    dset_img_train = hf.create_dataset('train_images', shape=(0, 3, 224, 224), chunks=image_chunk_shape, maxshape=(None,3,224,224))
    dset_label_train = hf.create_dataset('train_labels', shape=(0,), chunks=label_chunk_shape, maxshape=(None,))

with h5py.File('full_images.hdf5', "a") as f:
    train_img_ds = f["train_images"]
    train_label_ds = f['train_labels']
    num_rounds =  math.ceil(len(train_dataset) / BATCH_SIZE)
    for round_idx in range(num_rounds):
      start_idx = round_idx * BATCH_SIZE
      end_idx = min(start_idx + BATCH_SIZE, len(train_dataset))
      image_set = np.array([train_dataset[i][0] for i in range(start_idx, end_idx)])
      label_set = np.array([train_dataset[i][1] for i in range(start_idx, end_idx)])
      train_img_ds.resize((train_img_ds.shape[0] + len(image_set), 3, 224, 224))
      train_label_ds.resize((train_label_ds.shape[0] + len(label_set), ))
      train_img_ds[-len(image_set):] = image_set
      train_label_ds[-len(label_set):] = label_set

In [19]:
hdf5_file = 'full_images.hdf5'
trainDataLoader = DataLoader(dataset_h5(data_type='train', hdf5_path=hdf5_file), batch_size=BATCH_SIZE, shuffle=True)
testDataLoader = DataLoader(dataset_h5(data_type='val', hdf5_path=hdf5_file), batch_size=BATCH_SIZE, shuffle=True)

In [20]:
next(iter(trainDataLoader))[0].size(), next(iter(trainDataLoader))[1].size()

(torch.Size([128, 3, 224, 224]), torch.Size([128]))

In [21]:
resnet34 = models.resnet34(weights=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 79.8MB/s]


In [22]:
class CustomFC(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.fc1 = nn.Linear(in_features=in_features, out_features=out_features)

    def forward(self, x):
        return self.fc1(x), x

resnet34.fc = CustomFC(512,9)
resnet34 = resnet34.to(device)

In [26]:
def training_loop(model, dataloader_train, dataloader_test, model_name, n_epoch = 2):
    nf = f'model_best_{model_name}.pt'
    best_vloss = 1e+10
    best_acc = 0.0
    bntr = len(dataloader_train)
    bntst = len(dataloader_test)
    learning_rate = 1e-2
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_ce = nn.CrossEntropyLoss(reduction='mean')
    # n_epoch = 30

    for epoch in range(1, n_epoch+1):
        model.train()
        loss_train = 0.0
        acc_train = 0.0
        tot_train = 0.0
        for index, itms in enumerate(dataloader_train):
            print(index)
            x=itms[0]
            target=itms[1]
            x = x.to(device)
            target = target.to(device)
            outputs, _ = model(x)
            loss = loss_ce(outputs, target)
            # print(loss)
            acc_train += torch.sum(outputs.argmax(dim=1) == target).item()
            tot_train += target.size(0)


            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        acc_ratio_train = acc_train / tot_train

        model.eval()
        loss_val = 0.0
        acc_test = 0.0
        tot_test = 0.0

        with torch.no_grad():
            for itms in dataloader_test:
                x=itms[0]
                target=itms[1]
                x = x.to(device)
                target = target.to(device)
                outputs, _ = model(x)
                loss = loss_ce(outputs, target)

                loss_val += loss.item()
                acc_test += torch.sum(outputs.argmax(dim=1) == target).item()
                tot_test += target.size(0)
            acc_ratio_test = acc_test / tot_test
            if best_acc < acc_ratio_test:
                torch.save(model.state_dict(), nf)
                best_acc = acc_ratio_test
            # print(best_vloss, loss_val)



        if epoch == 1 or epoch%50 == 0:
            print(f'Epoch: {epoch}, Epoch train loss: {loss_train/bntr}, Acc train: {acc_ratio_train}')
            print(f'Epoch val loss: {loss_val/bntst}, Acc test: {acc_ratio_test}')
            # print(f'Accuracy: {correct/total*100}')

    return acc_ratio_test

In [27]:
training_loop(resnet34, trainDataLoader, testDataLoader, model_name = 'resnet34', n_epoch = 200)

0


KeyboardInterrupt: 